"""

HDF5 XPCS FitLab - Script for analyzing X-ray Photon Correlation Spectroscopy (XPCS) data from HDF5 files.

Author: Marilina Cathcarth [mcathcarth@gmail.com]

Version: 4.0

Date: October 12, 2023

Important Note:

This script relies on functions defined in the 'XPCS_functions.py' file. Please ensure that 'XPCS_functions.py' is located in the same directory as this script for proper execution.

**Dependencies:**

- Python (version >= 3.6)
- h5py (to work with HDF5 files)
- numpy (for numerical calculations)
- scipy (for curve fitting)
- scikit-learn (for calculating R2 score)
- matplotlib (for data visualization)
- qtpy (for GUI-based directory selection)
- pandas (for data manipulation)

**Installation:**

1. Make sure you have Python 3.6 or later installed. If not, download and install Python from [Python Downloads](https://www.python.org/downloads/).

2. Install the required packages using pip. Open a terminal or command prompt and run the following command:

    pip install h5py numpy scipy scikit-learn matplotlib qtpy pandas


3. Place the 'XPCS_functions.py' file in the same directory as this script to enable its functions for proper execution.

4. You're now ready to run the script for your XPCS analysis.

**Note:** If you encounter any issues, please ensure that all dependencies are correctly installed, and the 'XPCS_functions.py' file is in the same directory.

Make sure to follow these instructions for successful execution of your XPCS analysis script.

"""

In [2]:
import os
from XPCS_functions import select_synchrotron, select_directory, generate_base_name
import sys
from XPCS_functions import initialize_error_and_success_counters
from XPCS_functions import process_sirius_data, process_sirius_new_data, process_aps_data
import h5py
from XPCS_functions import initialize_data_for_parameter_averages, initialize_plot, initialize_data_for_derived_params
import pandas as pd
import matplotlib.pyplot as plt
import re
import numpy as np
from XPCS_functions import fit_model, fit_model_with_constraints, single_exponential, stretched_exponential, cumulants_model
import math
from XPCS_functions import calculate_relaxation_and_diffusion
from XPCS_functions import add_data_to_table, write_dat_file
from XPCS_functions import update_parameter_tracking_r
from XPCS_functions import plot_data_and_curves, fit_linear_model, configure_subplot, calculate_average_parameter_values
from XPCS_functions import print_summary, generate_fit_results_tables

# Default directory (can be left empty)
#directory = '/home/mac/research/experimental/test_XPCS/HDF5_XPCS/'
directory = ''

# Get the selected synchrotron
selected_synchrotron = select_synchrotron()

# Exit if no directory selected
if selected_synchrotron is None:
    print("User closed the window.")
    sys.exit()

# Call the select_directory function only if the directory is not defined
if not directory:
    directory = select_directory()

# Get a list of .hdf5 files in the directory
hdf5_files = [file for file in os.listdir(directory) if file.endswith('.hdf5')]

# Exit if no .hdf5 files are found
if not hdf5_files:
    print("No .hdf5 files found in the directory.")
    sys.exit()
    
#--------------------- Initializations ---------------------#

# Initialize error and success counters using the dictionary
counters = initialize_error_and_success_counters()

# Initialize empty lists for table data
table_data = {
    "single": [],     # Empty list for single table data
    "stretched": [],  # Empty list for stretched table data
    "cumulants": []   # Empty list for cumulants table data
}

#################################
### Loop over each .hdf5 file ###
#################################

for hdf5_file in hdf5_files:
    file_path = os.path.join(directory, hdf5_file)

    counters['hdf5_files'] += 1  # +++ Increment the hdf5_file counter +++

    # Generate the appropriate base name for output file naming based on selected synchrotron
    base_name = generate_base_name(selected_synchrotron, hdf5_file)
    
    #**************************************************************#
    # Obtain g2 vs. t data for each q value and create a DataFrame #
    #**************************************************************#
    
    # Process 'Sirius' data and get the DataFrame
    if selected_synchrotron == 'Sirius':
        dataset_df = process_sirius_data(file_path)
        
    # Process 'Sirius new' data and get the DataFrame
    elif selected_synchrotron == 'Sirius new':
        dataset_df = process_sirius_new_data(file_path)
    
    # Process 'APS' data and get the DataFrame
    elif selected_synchrotron == 'APS':
        dataset_df = process_aps_data(file_path)
    
    # Check if the DataFrame is None, indicating a failure in data processing
    if dataset_df is None:
        sys.exit()        # Terminate the program to handle the error or exception
        
    # +++ Increment the q_values counter +++
    q_count = dataset_df.shape[1] - 1       # Subtract 1 to exclude the 't' column
    counters['q_values'] += q_count                  
    
    #***** Save the data to a .dat file (CSV format with tab delimiter) *****#
            
    # Specify the output file path
    output_dataframe = os.path.join(directory, f"{base_name}_export_DataFrame.dat")
    
    # Check if the 'lineterminator' argument is available
    if 'lineterminator' in dir(pd.DataFrame.to_csv):
        line_terminator_arg = 'lineterminator'
    else:
        line_terminator_arg = 'line_terminator'

    # Then, use line_terminator_arg in the to_csv method
    dataset_df.to_csv(output_dataframe, sep='\t', index=False, **{line_terminator_arg: '\n'}, header=True)
    
    #--------------------- Initializations ---------------------#
    
    # Initialize data structures for parameter averages
    parameter_averages_data = initialize_data_for_parameter_averages()
    
    # Call the function to initialize the plot
    fig, ax_single, ax_stretched, ax_cumulants, ax_linear, cmap, lines_labels_dict = initialize_plot(q_count)

    # Initialize empty DataFrame and lists for relax_rate vs q^2 values
    derived_params = initialize_data_for_derived_params()
    
    ##############################
    ### Loop over each q value ###
    ##############################
    
    for i, q_column in enumerate(dataset_df.columns[1:], 1):
        
        # Extract the q value from the column name
        match = re.search(r"q=(\d+\.\d+)", q_column)
        if match:
            q_value = float(match.group(1))

        # Get the 't' and 'g2' data for the current q value
        t = dataset_df['# t']
        g2 = dataset_df[q_column]
    
        # Create the output filename
        output_name = f"{base_name}_export_{chr(ord('a')+i)}.dat"
        
        # Generate the complete output file path
        output_path = os.path.join(directory, output_name)
        
        ########################################################
        ###################### Fit models ######################
        ########################################################
            
        #------------------------------------------------------#
        #---------- Fit the Single Exponential Model ----------#
        #------------------------------------------------------#
            
        # Calculate the initial guess for parameters A and B:
        A0 = np.mean(g2[-5:])     # Average of the last 5 points
        B0 = np.mean(g2[:2]) - 1  # Average of the first 2 points minus 1

        # Calculate the initial guess for parameter C:
        # Obtain y
        y = (np.mean(g2[:2]) + np.mean(g2[-5:])) / 2
        # Find the closest x value in the experimental curve
        closest_index = np.abs(g2 - y).argmin()
        closest_x = t[closest_index]
        # Obtain C0:
        C0 = 1 / closest_x

        # Initial guess for parameters A, B, and C:
        initial_params_single = [A0, B0, C0]
        
        ## Fit the curve g2 = A + B * exp(-2C * t)
    
        fit_params_single, fitted_curve_single, r2_single  = fit_model_with_constraints(t, g2, single_exponential, initial_params_single)     
        fit_params_single0, fitted_curve_single0, r2_single0 = fit_model(t, g2, single_exponential, initial_params_single)

        if math.isnan(r2_single):
            # Add the filename to the list if fitting fails 
            counters['failure'] += 1        # +++ Increment the failure counter +++
            counters['failed_files'].append(f"{base_name} - q = {q_value} A^-1")

        # Calculate Relaxation time and Diffusion coefficient for Single Exponential
        relax_time_single, diffusion_coef_single = calculate_relaxation_and_diffusion(fit_params_single, q_value)
                      
        #***************************************************#
        # Add the data for (relax_rate vs q^2) in the lists #
        #***************************************************#
            
        derived_params["q_square"].append(q_value**2)
        derived_params["relax_rate"].append(fit_params_single[2])
            
        #***************************************************#
            
        #-------------------------------------------------------#
        #--------- Fit the Stretched Exponential model ---------#
        #-------------------------------------------------------#

        # Initial guess for parameter gamma
        gamma0 = 1

        # Use the parameters A, B, C from the Single Exponential fit and gamma0 as initial guess
        initial_params_stretched = [*fit_params_single, gamma0]
            
        ## Fit the curve g2 = A + B * exp(-2C * t)**gamma
                
        fit_params_stretched, fitted_curve_stretched, r2_stretched = fit_model_with_constraints(t, g2, stretched_exponential, initial_params_stretched)
        fit_params_stretched0, fitted_curve_stretched0, r2_stretched0 = fit_model(t, g2, stretched_exponential, initial_params_stretched)
        
        if math.isnan(r2_stretched):
            # Create the filename string
            failed_file = f"{base_name} - q = {q_value} A^-1"
            # Add the filename to the list if it's not already present
            if failed_file not in counters['failed_files']:
                counters['failure'] += 1       # +++ Increment the failure counter +++
                counters['failed_files'].append(failed_file)

        # Calculate Relaxation time and Diffusion coefficient for Stretched Exponential
        relax_time_stretched, diffusion_coef_stretched = calculate_relaxation_and_diffusion(fit_params_stretched, q_value)
            
        #-------------------------------------------------------#
        #--------- Analysis by the method of Cumulants ---------#
        #-------------------------------------------------------#
            
        # Initial guess for parameter C2
        C2_0 = 0.05*fit_params_single[2]**2
            
        # Use the parameters A, B, C from the Single Exponential fit and C2 as initial guess
        initial_params_cumulants = [*fit_params_single, C2_0]
            
        ## Fit the curve g2 = A + B * exp(-2C1 * t)*(1 + (1/2)* C2 * t**2)**2
            
        fit_params_cumulants, fitted_curve_cumulants, r2_cumulants = fit_model_with_constraints(t, g2, cumulants_model, initial_params_cumulants)
        #fit_params_cumulants0, fitted_curve_cumulants0, r2_cumulants0 = fit_model(t, g2, cumulants_model, initial_params_cumulants)
        
        if math.isnan(r2_cumulants):
            # Create the filename string
            failed_file = f"{base_name} - q = {q_value} A^-1"
            # Add the filename to the list if it's not already present
            if failed_file not in counters['failed_files']:
                counters['failure'] += 1       # +++ Increment the failure counter +++
                counters['failed_files'].append(failed_file)

        # Calculate Relaxation time and Diffusion coefficient for Cumulants
        relax_time_cumulants, diffusion_coef_cumulants = calculate_relaxation_and_diffusion(fit_params_cumulants, q_value)
        
        #-------------------------------------------------------#
        #-------------------------------------------------------#
        
        #******************************************************#
        #** Append data to the table and save to a .dat file **#
        #******************************************************#

        # for Single Exponential
        A_single = fit_params_single[0]
        B_single = fit_params_single[1]
        C_single = fit_params_single[2]
        # Append data to the table for Single Exponential
        add_data_to_table(table_data["single"], base_name, q_value, [A_single, B_single, C_single], relax_time_single, diffusion_coef_single, r2_single)

        # for Stretched Exponential
        A_stretched = fit_params_stretched[0]
        B_stretched = fit_params_stretched[1]
        C_stretched = fit_params_stretched[2]
        gamma_stretched = fit_params_stretched[3]
        # Append data to the table for Stretched Exponential
        add_data_to_table(table_data["stretched"], base_name, q_value, [A_stretched, B_stretched, C_stretched, gamma_stretched], relax_time_stretched, diffusion_coef_stretched, r2_stretched)
            
        # for Cumulants Model
        A_cumulants = fit_params_cumulants[0]
        B_cumulants = fit_params_cumulants[1]
        C1_cumulants = fit_params_cumulants[2]
        C2_cumulants = fit_params_cumulants[3]
        PDI_cumulants = C2_cumulants / C1_cumulants**2
        # Append data to the table for Cumulants Model
        add_data_to_table(table_data["cumulants"], base_name, q_value, [A_cumulants, B_cumulants, C1_cumulants, PDI_cumulants], relax_time_cumulants, diffusion_coef_cumulants, r2_cumulants)
            
        # Write data to .dat file for Single Exponential, Stretched Exponential and Cumulants
        write_dat_file(output_path, q_value, t, g2, A_single, B_single, C_single, A_stretched, B_stretched, C_stretched, gamma_stretched,
            r2_single, r2_stretched, relax_time_single, relax_time_stretched, diffusion_coef_single, diffusion_coef_stretched,
            A_cumulants, B_cumulants, C1_cumulants, C2_cumulants, r2_cumulants, relax_time_cumulants, diffusion_coef_cumulants)
            
        ########################################################
        ### Plot the experimental data and the fitted curves ###
        ########################################################

        color = cmap(i)  # Get color based on index
            
        # for Single Exponential
        plot_data_and_curves(ax_single, t, g2, A_single, B_single, fitted_curve_single, q_value, r2_single, lines_labels_dict, color, linestyle='-', model_type = "Single")

        # for Stretched Exponential
        plot_data_and_curves(ax_stretched, t, g2, A_stretched, B_stretched, fitted_curve_stretched, q_value, r2_stretched, lines_labels_dict, color, linestyle='--', model_type = "Stretched")
            
        # for Method of Cumulants
        plot_data_and_curves(ax_cumulants, t, g2, A_cumulants, B_cumulants, fitted_curve_cumulants, q_value, r2_cumulants, lines_labels_dict, color, linestyle='dotted', model_type = "Cumulants")
            
        #****************************************************************#
        # Calculation of the average parameters of the fits with R > 0.9 #
        #****************************************************************#
            
        # Check if r2_single is not NaN and if it's greater than 0.9 before updating parameter tracking.
        if not math.isnan(r2_single) and r2_single > 0.9:
            update_parameter_tracking_r(parameter_averages_data, 'single', r2_single, diffusion_coef_single)
        
        # Check if r2_stretched is not NaN and if it's greater than 0.9 before updating parameter tracking.
        if not math.isnan(r2_stretched) and r2_stretched > 0.9:
            update_parameter_tracking_r(parameter_averages_data, 'stretched', r2_stretched, diffusion_coef_stretched, gamma_stretched)

        # Check if r2_cumulants is not NaN and if it's greater than 0.9 before updating parameter tracking.
        if not math.isnan(r2_cumulants) and r2_cumulants > 0.9:
            update_parameter_tracking_r(parameter_averages_data, 'cumulants', r2_cumulants, diffusion_coef_cumulants, PDI_cumulants)         
            
        #****************************************************************#                        
            
        counters['success'] += 1  # +++ Increment the hdf5_file counter +++
        
        #print(parameter_averages_data)
        
    ###############################################################
    ################ Fit linear relax_rate vs q^2  ################
    ###############################################################
    
    #----------------------------------------------#
    #--------- Fit the curve C = D * q**2 ---------#
    #----------------------------------------------#
    
    try: 
        # Call the function to perform the fit
        D, pearson_r = fit_linear_model(derived_params["q_square"], derived_params["relax_rate"])

        # Create a color palette with the number of points
        n = len(derived_params["q_square"])
        #cmap = plt.get_cmap('gist_rainbow', n)
        #cmap = plt.get_cmap('turbo', n)
        
        # Plot the model data
        for i in range(n):
            color = cmap(i)
            line_data = ax_linear.plot(derived_params["q_square"][i], derived_params["relax_rate"][i], 'o', color=color, markersize=10, label="")[0]

        # Generate the linear fit line using the fitted value of D
        q_squared_fit = np.linspace(0, max(derived_params["q_square"]), 100)
        linear_fit_line = D * q_squared_fit
        
        ############################
        # Plot the linear fit line #
        ############################
        
        ax_linear.plot(q_squared_fit, linear_fit_line, linestyle='-', color='black', label=f"Linear Fit")

        # Set the axes to start from zero
        ax_linear.set_xlim(0,(max(derived_params["q_square"])+0.1*max(derived_params["q_square"])))
        ax_linear.set_ylim(0,(max(derived_params["relax_rate"])+0.1*max(derived_params["relax_rate"])))

    except ValueError as e:
        continue

    # Set labels
    ax_linear.set_xlabel("q^2 ($\AA^{-2}$)", fontsize=14)
    ax_linear.set_ylabel("Relaxation Rate (s^-1)", fontsize=14)
    ax_linear.set_title("Relax rate vs q^2", fontsize=16)
    ax_linear.legend()
    
    # Annotate the graph with slope (D) and Pearson correlation coefficient (pearson_r)
    slope_annotation = f"Slope (D) = {D:.2e}"
    r_annotation = f"Pearson r = {pearson_r:.4f}"
    
    # Get the coordinates of the graph label
    label_x, label_y = 0.02, ax_linear.get_ylim()[1]

    # Add the label in the upper-left corner without a border
    ax_linear.annotate(slope_annotation, (0.02, 0.90), xycoords='axes fraction', color="black", fontsize=12)

    # Add the annotations below the label
    ax_linear.annotate(r_annotation, (0.02, 0.85), xycoords='axes fraction', color="black", fontsize=12)

    ###############################################
    ### Configure the subplots for the 3 models ###
    ###############################################
    
    # Configure subplot for Single Exponential
    configure_subplot(ax_single, "Single Exponential", "Delay Time (s)", "(g2-base) / beta",
                      lines_labels_dict["exp_lines_single"],
                      lines_labels_dict["exp_labels_single"],
                      lines_labels_dict["fit_lines_single"],
                      lines_labels_dict["fit_labels_single"])
    
    # Calculate average parameter values for the 'single' model if R count is not zero
    if parameter_averages_data['single']['R count'] > 0:
        average_single_parameters = calculate_average_parameter_values(parameter_averages_data, 'single') 
        
        # Add average parameter values to Single Exponential subplot
        ax_single.text(0.65, 0.98, f"Av. Diff Coef (u2/s): {average_single_parameters['Diff_coef av']:.4f} ({average_single_parameters['Diff_coef std']:.4f})", 
                          transform=ax_single.transAxes, va='top', ha='right', fontsize=12)

    # Configure subplot for Stretched Exponential
    configure_subplot(ax_stretched, "Stretched Exponential", "Delay Time (s)", "(g2-base) / beta",
                      lines_labels_dict["exp_lines_stretched"],
                      lines_labels_dict["exp_labels_stretched"],
                      lines_labels_dict["fit_lines_stretched"],
                      lines_labels_dict["fit_labels_stretched"])
    
    # Calculate average parameter values for the 'stretched' model if R count is not zero
    if parameter_averages_data['stretched']['R count'] > 0:
        average_stretched_parameters = calculate_average_parameter_values(parameter_averages_data, 'stretched')
        
        # Add average parameter values to Stretched Exponential subplot
        ax_stretched.text(0.65, 0.98, f"Av. Diff Coef (u2/s): {average_stretched_parameters['Diff_coef av']:.4f} ({average_stretched_parameters['Diff_coef std']:.4f})", 
                          transform=ax_stretched.transAxes, va='top', ha='right', fontsize=12)
        ax_stretched.text(0.65, 0.94, f"Av. Gamma: {average_stretched_parameters['Gamma av']:.4f} ({average_stretched_parameters['Gamma std']:.4f})", 
                          transform=ax_stretched.transAxes, va='top', ha='right', fontsize=12)

    # Configure subplot for Cumulants model
    configure_subplot(ax_cumulants, "Cumulants", "Delay Time (s)", "(g2-base) / beta",
                      lines_labels_dict["exp_lines_cumulants"],
                      lines_labels_dict["exp_labels_cumulants"],
                      lines_labels_dict["fit_lines_cumulants"],
                      lines_labels_dict["fit_labels_cumulants"])
    
    # Calculate average parameter values for the 'cumulants' model if R count is not zero
    if parameter_averages_data['cumulants']['R count'] > 0:
        average_cumulants_parameters = calculate_average_parameter_values(parameter_averages_data, 'cumulants')
        
        # Add average parameter values to Cumulants subplot
        ax_cumulants.text(0.65, 0.98, f"Av. Diff Coef (u2/s): {average_cumulants_parameters['Diff_coef av']:.4f} ({average_cumulants_parameters['Diff_coef std']:.4f})", 
                          transform=ax_cumulants.transAxes, va='top', ha='right', fontsize=12)
        ax_cumulants.text(0.65, 0.94, f"Av. PDI: {average_cumulants_parameters['PDI av']:.4f} ({average_cumulants_parameters['PDI std']:.4f})", 
                          transform=ax_cumulants.transAxes, va='top', ha='right', fontsize=12)

    ###############################################

    # Save the figure to a PDF file
    pdf_name = f"{base_name}.pdf"
    output_pdf_path = os.path.join(directory, pdf_name)
    plt.tight_layout()
    plt.savefig(output_pdf_path, format='pdf')
    #plt.savefig(output_pdf_path, bbox_inches='tight', format='pdf')

    # Close the figure to free up memory
    plt.close(fig)
    
    # Sort the columns of combined_df based on q values
    dataset_df = dataset_df[['# t'] + sorted(dataset_df.columns[1:], key=lambda col_name: float(re.search(r'q=([0-9.]+)', col_name).group(1)))]

    # Save the DataFrame to a .dat file
    dataset_df.to_csv(output_dataframe, sep='\t', index=False)

plt.close()
    
# Print the counts, filed files and invalid files
print_summary(counters)

# Generate the new .dat files with the fit results tables for each model
generate_fit_results_tables(directory, table_data["single"], table_data["stretched"], table_data["cumulants"])

Total HDF5 files: 1
Total x files: 8
Total invalid files: 0
Successful fits: 8
Failed fits: 0
Fit results tables generated successfully.
